###  MicroGrad demo

In [11]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mycrograd_debug.engine_debug import Value
from mycrograd_debug.nn_debug import Neuron, Layer, MLP
from mycrograd_debug.drawviz_debug import draw_dot, draw_nn
from mycrograd_debug.util_debug import (
    debugPrint,
    print_my_params,
    print_all_values,
    backupParameters,
    restoreParameters,
)
import pprint
np.random.seed(1337)
random.seed(1337)
number_of_samples=4
number_of_iterations=100
pp = pprint.PrettyPrinter(indent=4)
debug_values = False


In [12]:
# make up a dataset

from sklearn.datasets import make_moons, make_blobs
X, y = make_moons(n_samples=number_of_samples, noise=0.1)
pp.pprint(X)
pp.pprint(y)
y = y*2 - 1 # make y be -1 or 1
pp.pprint(y)

array([[ 0.02183808,  0.35430742],
       [ 0.82773723,  0.0961832 ],
       [-1.09199463, -0.10965849],
       [ 2.1335573 ,  0.42420399]])
array([1, 0, 0, 1])
array([ 1, -1, -1,  1])


In [13]:
# initialize a model 
# model = MLP(2, [16, 16, 1]) # 2-layer neural network
model = MLP(2, [2, 1], lastReLU=True, debug_bw=False) # 2-layer neural network
# print(model)
# pp.pprint(model.parameters())
# pp.pprint(model.layers)

Xb, yb = X, y
inputs = [list(map(Value, xrow)) for xrow in Xb]
pp.pprint(inputs)


Module nn MLP: structure [2, 2, 1]
[   [   Value(name=v11594,layernumber=,neuronnumber=,weightnumber=,type=,data=0.0218380834585633, grad=0),
        Value(name=v11595,layernumber=,neuronnumber=,weightnumber=,type=,data=0.35430741854819914, grad=0)],
    [   Value(name=v11596,layernumber=,neuronnumber=,weightnumber=,type=,data=0.8277372309744037, grad=0),
        Value(name=v11597,layernumber=,neuronnumber=,weightnumber=,type=,data=0.09618319872606, grad=0)],
    [   Value(name=v11598,layernumber=,neuronnumber=,weightnumber=,type=,data=-1.0919946290642557, grad=0),
        Value(name=v11599,layernumber=,neuronnumber=,weightnumber=,type=,data=-0.1096584897551003, grad=0)],
    [   Value(name=v11600,layernumber=,neuronnumber=,weightnumber=,type=,data=2.133557297233384, grad=0),
        Value(name=v11601,layernumber=,neuronnumber=,weightnumber=,type=,data=0.42420399367691636, grad=0)]]


In [14]:
# loss function
def loss():
    
    # forward the model to get scores
    # scores = list(map(model, inputs))
    mymap=map(model, inputs)
    # print("mymap:")
    # pp.pprint(mymap)
    scores = list(mymap)
    print("scores:")
    pp.pprint(scores)

    # svm "max-margin" loss
    losses = [(1 + -yi*scorei).relu() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    total_loss = data_loss + reg_loss
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

# total_loss, acc = loss()
# print(total_loss, acc)

In [15]:


# optimization
makeimg=False
for k in range(number_of_iterations):
    print("start step %d" %k)
    # pp.pprint(model.parameters())

    # forward
    total_loss, acc = loss()
    print("step %d loss calc" %k)
    if debug_values:
        print_all_values(total_loss)
    if makeimg:
        dot=draw_dot(total_loss)
        dot.render("images/opt_01_step%d_1loss" % k , format="svg", view=True)

    # backward
    model.zero_grad()
    print("step %d zero grad" %k)
    if debug_values:
        print_all_values(total_loss)
    if makeimg:
        dot=draw_dot(total_loss)
        dot.render("images/opt_01_step%d_2zero" % k , format="svg", view=True)

    total_loss.backward()
    print("step %d backward" %k)
    if debug_values:
        print_all_values(total_loss)
    if makeimg:
        dot=draw_dot(total_loss)
        dot.render("images/opt_01_step%d_3back" % k , format="svg", view=True)
    
    # update (sgd)
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    # if k % 1 == 0:
    #     print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")
    print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")


start step 0
scores:
[   Value(name=v11615,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.016436428967818627, grad=0),
    Value(name=v11629,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=-0.1346183875256753, grad=0),
    Value(name=v11643,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.17786782634182813, grad=0),
    Value(name=v11657,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=-0.3548313015504008, grad=0)]
step 0 loss calc
step 0 zero grad
step 0 backward
step 0 loss 1.0955140025642771, accuracy 50.0%
start step 1
scores:
[   Value(name=v11720,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.05078620433112108, grad=0),
    Value(name=v11734,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.0, grad=0),
    Value(name=v11748,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.040733300229352204, grad=0),
    Value(name=v11762,layernumber=L2,neuronnumber=N1,weightnumber=,type=a,data=0.0, grad=0)]
step 1 loss calc
st